Modelo LSTM de Google Colab

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# import os

# # Ruta del nuevo archivo CSV
# csv = '/content/drive/MyDrive/archive/appliance_usage_data2.csv'

# # Ajustar la longitud de la secuencia
# sequence_length = 10

# # Hiperparámetros de la red LSTM
# num_features = 1
# epochs = 10

# # Verificar la disponibilidad de GPU
# if tf.test.gpu_device_name():
#     print('GPU encontrada:', tf.test.gpu_device_name())
# else:
#     print('No se encontró GPU. Asegúrate de habilitar la GPU en "Entorno de ejecución".')

# # Comprobar si el archivo del modelo existe
# model_filename = 'modelo.h5'
# if os.path.exists(model_filename):
#     print("Cargando modelo existente")
#     # Si el archivo del modelo existe, cargar el modelo
#     model = tf.keras.models.load_model(model_filename)
# else:
#     # Si el archivo del modelo no existe, crear un nuevo modelo
#     with tf.device('/device:GPU:0'):
#         model = Sequential()
#         model.add(LSTM(50, input_shape=(sequence_length, num_features)))
#         model.add(Dense(num_features))
#         model.compile(optimizer='adam', loss='mse')

# # Crear un DataFrame vacío para almacenar los datos
# df = pd.read_csv(csv)

# # Convertir la columna de timestamp en datetime
# df['Fecha'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')

# # Establecer la columna de timestamp como el índice del dataframe
# df = df.set_index('Fecha')

# # Normalizar los datos de potencia
# scaler = StandardScaler()
# df['Watts usados'] = scaler.fit_transform(df['Watts usados'].values.reshape(-1, 1))

# with tf.device('/device:GPU:0'):
#     # Crear secuencias de datos para entrenar la red LSTM
#     data = []
#     for i in range(len(df) - sequence_length):
#         data.append(df['Watts usados'].iloc[i:i+sequence_length])

#     # Convertir la lista de secuencias en un array numpy
#     data = np.array(data)

#     # Asegúrate de que la cantidad de datos de entrenamiento sea un múltiplo de sequence_length
#     train_size = len(data) - (len(data) % sequence_length)
#     x_train = data[:train_size, :-1]
#     y_train = data[:train_size, -1]
#     x_train = x_train.reshape(-1, sequence_length, num_features)

#     # Asegúrate de que x_train y y_train tengan la misma longitud
#     min_length = min(len(x_train), len(y_train))
#     x_train = x_train[:min_length]
#     y_train = y_train[:min_length]

#     # Entrenar el modelo
#     model.fit(x_train, y_train, epochs=epochs)

#     # Guardar el modelo entrenado
#     model.save('modelo.h5')

# # Cargar el modelo en el futuro
# modelo_cargado = tf.keras.models.load_model('modelo.h5')


Modelo con clasificación binaria

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Paso 1: Cargar y preparar los datos
data = pd.read_csv('appliance_usage_data1.csv')
X = data[['Fecha', 'Hora', 'Temperatura']]
y = data['Reporte de fallo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 2: Preprocesamiento de datos
X_train['Fecha'] = pd.to_datetime(X_train['Fecha'])
X_train['DiaDelAño'] = X_train['Fecha'].dt.dayofyear
X_train = X_train.drop(columns=['Fecha'])
scaler = StandardScaler()
X_train[['Hora', 'DiaDelAño']] = scaler.fit_transform(X_train[['Hora', 'DiaDelAño']])

# Paso 3: Crear y entrenar el modelo de clasificación
model = keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Paso 4: Evaluar el modelo
X_test['Fecha'] = pd.to_datetime(X_test['Fecha'])
X_test['DiaDelAño'] = X_test['Fecha'].dt.dayofyear
X_test = X_test.drop(columns=['Fecha'])
X_test[['Hora', 'DiaDelAño']] = scaler.transform(X_test[['Hora', 'DiaDelAño']])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida en el conjunto de prueba: {loss}')
print(f'Precisión en el conjunto de prueba: {accuracy}')

KeyError: "['Temperatura'] not in index"

In [1]:
# Importa las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Cargando los datos desde el archivo CSV
data = pd.read_csv("appliance_usage_data21.csv")

# Dividiendo los datos en características (X) y la variable objetivo (y)
X = data[["Hora", "Watts usados", "Uso intensivo", "Porcentaje de uso"]]
y = data["Reporte de fallo"]

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando las características, lo cual es importante para SVM
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creando un modelo SVM
model = SVC(kernel='linear', C=1.0)  # Puedes ajustar el kernel y el parámetro C según sea necesario

# Entrenando el modelo en el conjunto de entrenamiento
model.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calculando la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

datasets = [
    'appliance_usage_data1.csv',
    'appliance_usage_data2.csv',
    'appliance_usage_data3.csv',
    'appliance_usage_data4.csv',
    'appliance_usage_data5.csv',
    'appliance_usage_data6.csv',
    'appliance_usage_data7.csv',
    'appliance_usage_data8.csv',
    'appliance_usage_data9.csv',
    'appliance_usage_data10.csv',
    'appliance_usage_data11.csv',
    'appliance_usage_data12.csv',
    'appliance_usage_data13.csv',
    'appliance_usage_data14.csv',
    'appliance_usage_data15.csv',
    'appliance_usage_data16.csv',
    'appliance_usage_data17.csv',
    'appliance_usage_data18.csv',
    'appliance_usage_data19.csv',
    'appliance_usage_data20.csv',
]

# Crear el modelo de clasificación
model = keras.Sequential([
        layers.Input(shape=(2,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Paso 1: Cargar y preparar los datos
for dataset in datasets:
    data = pd.read_csv(dataset)
    X = data[['Fecha', 'Hora']]
    
    y = data['Reporte de fallo']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Paso 2: Preprocesamiento de datos
    X_train['Fecha'] = pd.to_datetime(X_train['Fecha'])
    X_train['DiaDelAño'] = X_train['Fecha'].dt.dayofyear
    X_train = X_train.drop(columns=['Fecha'])
    
    scaler = StandardScaler()
    X_train[['Hora', 'DiaDelAño']] = scaler.fit_transform(X_train[['Hora', 'DiaDelAño']])

    
    model.fit(X_train, y_train, epochs=10, batch_size=32)

# Guardar el modelo en un archivo h5 después de entrenar con todos los archivos
model.save('model.h5')

# Paso 4: Evaluar el modelo con el último conjunto de datos
X_test['Fecha'] = pd.to_datetime(X_test['Fecha'])
X_test['DiaDelAño'] = X_test['Fecha'].dt.dayofyear
X_test = X_test.drop(columns=['Fecha'])

X_test[['Hora', 'DiaDelAño']] = scaler.transform(X_test[['Hora', 'DiaDelAño']])

loss, accuracy = model.evaluate(X_test, y_test)

print(f'Pérdida en el conjunto de prueba: {loss}')
print(f'Precisión en el conjunto de prueba: {accuracy}')

In [3]:
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score

# Cargar el modelo desde el archivo .h5
model = load_model('modelo_clasificacion_fallo1.h5')

# Datos de entrada
# Abriendo el archivo CSV
data = pd.read_csv('nuevos_datos1.csv')

# Normalizar los datos
scaler = MinMaxScaler()
data['Watts usados'] = scaler.fit_transform(data['Watts usados'].values.reshape(-1, 1))

# Preparar los datos para la predicción
X_pred = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values

# Hacer la predicción
y_pred = model.predict(X_pred)

print('Predicciones:', y_pred)

# Convertir las probabilidades en etiquetas de clase
y_pred = (y_pred >= 0.5).astype(int)

# Los valores reales están en la columna 'Reporte de fallo'
y_real = data['Reporte de fallo'].values

# Calcular la precisión y exactitud
accuracy = accuracy_score(y_real, y_pred)
precision = precision_score(y_real, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')

# Guardando las predicciones en un archivo de texto.
with open('predicciones_binarias.txt', 'w') as f:
    for i in range(len(y_pred)):
        f.write(f'Predicción: {y_pred[i]} - Valor real: {y_real[i]}\n')

15625/15625 [==============================] - 22s 1ms/step
Predicciones: [[1.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 ...
 [1.3752777e-10]
 [1.0000000e+00]
 [3.8580250e-29]]
Accuracy: 1.0
Precision: 1.0


Probado de otra forma el modelo de clasificación binaria.

In [ ]:
# import pandas as pd
# import numpy as np
# import pickle
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Cargar los datos desde el archivo CSV
# data = pd.read_csv('appliance_usage_data2.csv')

# # Normalizar los datos directamente en el DataFrame
# data['Watts usados'] = (data['Watts usados'] - data['Watts usados'].min()) / (data['Watts usados'].max() - data['Watts usados'].min())

# # Dividir los datos en conjuntos de entrenamiento y prueba
# X = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values
# y = data['Watts usados'].values  # Valores normalizados entre 0 y 1

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Crear el modelo de regresión (ya que ahora y es una variable continua)
# model = Sequential()
# model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(1, activation='linear'))  # Linear activation for regression
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# # Entrenar el modelo
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# # Evaluar el modelo
# mse, mae = model.evaluate(X_test, y_test)
# print(f'Mean Squared Error: {mse}')
# print(f'Mean Absolute Error: {mae}')

# # Hacer predicciones
# y_pred = model.predict(X_test)

# # Comparando los valores reales con las predicciones.
# print('Valores reales:', y_test[:5])
# print('Predicciones:', y_pred[:5])

# # Guardando las predicciones.
# np.savetxt('predicciones_binarias_normalizadas.txt', y_pred)

# # Guardar el modelo en un archivo
# with open('modelo_regresion.pkl', 'wb') as file:
#     pickle.dump(model, file)